In [1]:
!pip install transformers
!pip install tensorflow
!pip install torch

import numpy as np 
from pathlib import Path
from transformers import pipeline, AutoTokenizer

#Je défini la Moyenne et j'utilise un modèle
def MoyenneE(base_path, output_path):
    model_n = "bhadresh-savani/bert-base-go-emotion"
    tokenizer = AutoTokenizer.from_pretrained(model_n)
    classifier = pipeline("text-classification", model=model_n, return_all_scores=True)

    #Parcourt chaque dossier
    for namedir in base_path.iterdir():
        if namedir.is_dir():
            results = []
            for txt_file in namedir.glob("*.txt"):
                with open(txt_file, encoding="utf-8") as f:
                    text = f.read()

                #Tokenize le texte
                tokens = tokenizer.tokenize(text)

                #Liste pour découper le texte
                blocks = []
                for i in range(0, len(tokens), 500):
                    block_tok = tokens[i:i+500]
                    block_text = tokenizer.convert_tokens_to_string(block_tok)
                    blocks.append(block_text)

                #Variable pour stocker kes scores et les étiquettes
                emotion_score = None
                labels = None

                #Parcours chaque bloc et prédit les émotions
                for block in blocks:
                    preds = classifier(block)[0]
                    scores = np.array([p["score"] for p in preds])
                    if emotion_score is None:
                        emotion_score = scores
                        labels = [p["label"] for p in preds]
                    else:
                        emotion_score += scores

                #Moyenne des scores
                emotion_score /= len(blocks)
                av = dict(zip(labels, emotion_score.tolist()))
                results.append((txt_file.stem, av))

            #Création du fichier
            output_f = output_path / f"{namedir.name}MEmotion.txt"
            with open(output_f, 'w', encoding="utf-8") as f:
                for filename, prediction in results:
                    f.write(f"{filename}:\n")
                    for label, score in prediction.items():
                        f.write(f"{label}:{score:.4f}\n")
                    f.write("\n")

if __name__ =="__main__":
    base_path = Path.home() / "PROJECT"/"data"/"clean"
    figures_path = Path.home() /"PROJECT"/"figures"/"MEmotions"
    figures_path.mkdir(parents=True, exist_ok=True)
    MoyenneE(base_path, figures_path)                                                 



  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl (375.9 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\AyaKu\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



C:\Users\AyaKu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W0519 14:01:45.879000 25692 torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
Device set to use cpu
C:\Users\AyaKu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for